In [1]:
# https://learnmeabitcoin.com/technical/transaction/

In [1]:
import os
import json
import hashlib 

In [2]:
class Prevout:
    def __init__(self, scriptpubkey, scriptpubkey_asm, scriptpubkey_type, scriptpubkey_address, value):
        self.scriptpubkey = scriptpubkey
        self.scriptpubkey_asm = scriptpubkey_asm
        self.scriptpubkey_type = scriptpubkey_type
        self.scriptpubkey_address = scriptpubkey_address
        self.value = value

    def __str__(self):
        return f"Prevout: scriptpubkey={self.scriptpubkey}, scriptpubkey_asm={self.scriptpubkey_asm}, scriptpubkey_type={self.scriptpubkey_type}, scriptpubkey_address={self.scriptpubkey_address}, value={self.value}"

class Vin:
    def __init__(self, txid, vout, prevout, scriptsig, scriptsig_asm, witness, is_coinbase, sequence):
        self.txid = txid
        self.vout = vout
        self.prevout = prevout
        self.scriptsig = scriptsig
        self.scriptsig_asm = scriptsig_asm
        self.witness = witness
        self.is_coinbase = is_coinbase
        self.sequence = sequence

    def __str__(self):
        return f"Vin: txid={self.txid}, vout={self.vout}, prevout={self.prevout}, scriptsig={self.scriptsig}, scriptsig_asm={self.scriptsig_asm}, witness={self.witness}, is_coinbase={self.is_coinbase}, sequence={self.sequence}"

class Vout:
    def __init__(self, scriptpubkey, scriptpubkey_asm, scriptpubkey_type, scriptpubkey_address, value):
        self.scriptpubkey = scriptpubkey
        self.scriptpubkey_asm = scriptpubkey_asm
        self.scriptpubkey_type = scriptpubkey_type
        self.scriptpubkey_address = scriptpubkey_address
        self.value = value

    def __str__(self):
        return f"Vout: scriptpubkey={self.scriptpubkey}, scriptpubkey_asm={self.scriptpubkey_asm}, scriptpubkey_type={self.scriptpubkey_type}, scriptpubkey_address={self.scriptpubkey_address}, value={self.value}"

class Txn:
    def __init__(self, txid, version, locktime, vin, vout, fee, weight):
        self.txid = txid
        self.version = version
        self.locktime = locktime
        self.vin = vin
        self.vout = vout
        self.fee = fee
        self.fee_rate = fee / weight
        self.weight = weight

    def __str__(self):
        vin_str = "\n".join(str(v) for v in self.vin)
        vout_str = "\n".join(str(v) for v in self.vout)
        return f"Txid: {self.txid}, version: {self.version}, locktime: {self.locktime}, fee: {self.fee}, weight: {self.weight}\nVin:\n{vin_str}\nVout:\n{vout_str}"


In [3]:
def read_directory(directory):
    """
    Read transaction data from JSON files in the specified directory.

    Parameters:
    - directory (str): The directory containing JSON files.

    Returns:
    - dict: A dictionary where keys are filenames (without the .json extension) and values are the corresponding transaction data.
    """
    tx_data = {}

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isfile(file_path) and filename.endswith(".json"):
            try:
                # Read JSON data from the file
                with open(file_path, "r") as file:
                    json_data = json.load(file)
                    # Parse transaction data using parse_transaction function
                    transaction = parse_transaction(filename[:-5], json_data)
                    # Store transaction object in the dictionary with filename (without extension) as key
                    tx_data[filename[:-5]] = transaction
            except (FileNotFoundError, json.JSONDecodeError) as e:
                print(f"Error reading file {filename}: {e}")

    return tx_data
    
def parse_transaction(filename, json_data):
    txid = filename
    version = json_data.get("version")
    locktime = json_data.get("locktime")
    
    vin = [Vin(
        txid=input_data.get("txid"),
        vout=input_data.get("vout"),
        prevout=Prevout(**input_data.get("prevout", {})),
        scriptsig=input_data.get("scriptsig"),
        scriptsig_asm=input_data.get("scriptsig_asm"),
        witness=input_data.get("witness", []),
        is_coinbase=input_data.get("is_coinbase", False),
        sequence=input_data.get("sequence")
    ) for input_data in json_data.get("vin", [])]

    vout = [Vout(
        scriptpubkey=output_data.get("scriptpubkey"),
        scriptpubkey_asm=output_data.get("scriptpubkey_asm"),
        scriptpubkey_type=output_data.get("scriptpubkey_type"),
        scriptpubkey_address=output_data.get("scriptpubkey_address"),
        value=output_data.get("value")
    ) for output_data in json_data.get("vout", [])]

    fee = calculate_fee(vin, vout)
    weight = calculate_txn_weight(json_data)

    return Txn(txid, version, locktime, vin, vout, fee, weight)

def calculate_fee(vin, vout):
    """
    Calculate the fee rate for a given transaction.

    Parameters:
    - transaction (Transaction): The transaction object.

    Returns:
    - float: The fee rate of the transaction.
    """
    total_input_value = sum(vin.prevout.value for vin in vin)
    total_output_value = sum(vout.value for vout in vout)
    fee = total_input_value - total_output_value
    return fee

def calculate_txn_weight(json_data):
    """
    Calculate the weight of a given transaction.

    Parameters:
    - json_data (dict): The JSON data representing the transaction.

    Returns:
    - int: The weight of the transaction.
    """
    # Calculate the size of non-witness data
    non_witness_size = sum(len(json.dumps(item)) for item in [
        json_data.get("version"),
        json_data.get("locktime"),
        json_data.get("vin"),
        json_data.get("vout")
    ])

    # Calculate the size of witness data
    witness_size = sum(len(json.dumps(item)) for item in json_data.get("vin", []) if item.get("witness"))

    # Calculate the weight
    weight = 4 * non_witness_size + witness_size
    return weight


In [4]:
# For legacy transactions the weight would be the bytes of the transaction on disk 
# times 4.   For segwit you weight the version, the inputs, outputs and the locktime 
# bytes times four while discounting the new segwit parts(the marker, the flag and 
# the witness) to just 1x.  Taproot would be the same as segwit with the new fields 
# discounted.  Segwit had the side effect of increasing the blocksize to around 2mb 
# on disk (4 weight units) because typically segwit transactions had roughly half 
# the bytes in the witness.   The ordinal folks and their transaction have much 
# more witness data which is why recenlty we have seen block much closer to 4 mb on 
# disk and 4 weight units because in the ordinal transaction a much higher 
# percentage of the transaction is in the witness

In [5]:
transactions = read_directory("mempool")

In [6]:
# counter = 0

# for key, value in transactions.items():
#     print(key, value)
#     counter += 1
#     if counter == 3:
#         break

In [25]:
from typing import List
from ecdsa import VerifyingKey
from ecdsa.curves import SECP256k1
from ecdsa.util import sigdecode_der
import ecdsa

def get_p2wpkh_scriptcode(pubkey_hash: bytes) -> bytes:
    """Given a JSON utxo object, extract the public key hash from the output script"""
    # Extract the public key hash from the UTXO object
    # pubkey_hash = bytes.fromhex(utxo['scriptPubKey']['hex'])[2:]

    # Assemble the P2WPKH script code
    scriptcode = (25).to_bytes(1, 'big')  # Script length
    scriptcode += bytes.fromhex("76a9")  # OP_DUP OP_HASH160
    scriptcode += len(pubkey_hash).to_bytes(1, 'big')  # Pubkey hash length
    scriptcode += pubkey_hash  # Pubkey hash
    scriptcode += bytes.fromhex("88ac")  # OP_EQUALVERIFY OP_CHECKSIG

    # print(f"scriptcode {scriptcode.hex()}")
    return scriptcode

def get_commitment_hash(outpoint: bytes, scriptcode: bytes, value: int, outputs: List[bytes], locktime: bytes, sequence: bytes) -> bytes:
    """Given a transaction outpoint, scriptcode, value, and outputs, compute the commitment hash"""
    def dsha256(data: bytes) -> bytes:
        return hashlib.new("sha256", hashlib.new("sha256", data).digest()).digest()

    # Version (4-byte little endian)
    version = b'\x02\x00\x00\x00'

    # All TX input outpoints (only one in our case)
    hash_prevouts = dsha256(outpoint)
    # hash_prevouts = dsha256(b"".join(outpoint))

    # outpoints = b"".join(outpoint)
    
    # Sequence of output being spent (always default for us)
    # sequence = b"\xff\xff\xff\xff"
    hash_sequence = dsha256(sequence)

    # Value of output being spent (8-byte little endian)
    value_bytes = value.to_bytes(8, byteorder="little")

    # All TX outputs
    hash_outputs = dsha256(b"".join(outputs))

    # Locktime (always default for us)
    # locktime = b"\x00\x00\x00\x00"

    # SIGHASH_ALL (always default for us)
    sighash_type = b"\x01\x00\x00\x00"

    # Concatenate all the components
    commitment_data = version + hash_prevouts + hash_sequence + outpoint + scriptcode + value_bytes + sequence + hash_outputs + locktime + sighash_type

    return dsha256(commitment_data)

def verify(pub: bytes, msg: bytes, sig: bytes) -> bool:
    """Given a public key, message, and signature, verify the signature"""
    vk = VerifyingKey.from_string(pub, curve=SECP256k1)
    try:
        # Attempt to verify the signature
        vk.verify_digest(sig, msg, sigdecode=sigdecode_der)
        return True
    except ecdsa.BadSignatureError:
        # If the verification fails, return False
        print("verification fails")
        return False

def assemble_transaction(inputs: List[bytes], outputs: List[bytes], witnesses: List[bytes]) -> str:
    """Given arrays inputs, outputs, witnesses, assemble transaction and serialize for broadcast"""
    # Version
    version = (2).to_bytes(4, "little")
    flags = bytes.fromhex("0001")

    # Inputs
    num_inputs = len(inputs).to_bytes(1, "little")
    tx_inputs = b"".join(inputs)

    # Outputs
    num_outputs = len(outputs).to_bytes(1, "little")
    tx_outputs = b"".join(outputs)

    # Witnesses
    num_witnesses = len(witnesses).to_bytes(1, "little")
    witness_bytes = b""
    for witness in witnesses:
        witness_bytes += len(witness).to_bytes(1, "little") + witness

    # Locktime
    locktime = bytes.fromhex("00000000")

    # Assemble the transaction
    transaction = version + flags + num_inputs + tx_inputs + num_outputs + tx_outputs + num_witnesses + witness_bytes + locktime

    return transaction.hex()

def get_txid(inputs: List[bytes], outputs: List[bytes]) -> str:
    """Given arrays of inputs and outputs, compute the hash of the transaction without witnesses"""
    # Version
    version = (2).to_bytes(4, "little")

    # Inputs
    num_inputs = len(inputs).to_bytes(1, "little")
    tx_inputs = b"".join(inputs)

    # Outputs
    num_outputs = len(outputs).to_bytes(1, "little")
    tx_outputs = b"".join(outputs)

    # Locktime
    locktime = bytes.fromhex("00000000")

    # Compute hash of transaction without witnesses
    raw_transaction = version + num_inputs + tx_inputs + num_outputs + tx_outputs + locktime
    txid = hashlib.sha256(hashlib.sha256(raw_transaction).digest()).digest()[::-1].hex()

    return txid

# Given an output script and value (in satoshis), return a serialized transaction output
def output_from_options(script: bytes, value: int) -> bytes:
    """Given an output script and value (in satoshis), return a serialized transaction output"""
    # Construct the serialized transaction output
    output_data = bytearray()

    # 8-byte little-endian integer representing the value (amount) of the output
    output_data.extend(value.to_bytes(8, byteorder='little'))

    # Length of the output script
    output_data.append(len(script))

    # Output script
    output_data.extend(script)

    return bytes(output_data)

def input_from_utxo(txid: bytes, index: int) -> bytes:
    """Given an outpoint, return a serialized transaction input spending it"""
    # Construct the input spending the given outpoint
    input_data = bytearray()

    # Transaction ID (little-endian)
    input_data.extend(txid[::-1])

    # Output index (4-byte little-endian integer)
    input_data.extend(index.to_bytes(4, byteorder='little'))

    # Script length (as it's hardcoded, 0x00 since scriptSig is empty)
    input_data.append(0x00)

    # Sequence (4-byte little-endian integer)
    input_data.extend((0xFFFFFFFF).to_bytes(4, byteorder='little'))

    # print(f"input_data {input_data.hex()}")
    return bytes(input_data)


In [29]:
# TEST ONE
count = 0
for filename, transaction in transactions.items():
    # print(filename, transaction)

    inpt = bytes.fromhex(transaction.vin[0].txid)[::-1] + int(transaction.vin[0].vout).to_bytes(4, byteorder='little')
    value_sats = transaction.vin[0].prevout.value

    script_code = get_p2wpkh_scriptcode(bytes.fromhex(transaction.vin[0].prevout.scriptpubkey)[2:])

    outputs = []
    for vout in transaction.vout:
        if vout.scriptpubkey_type == 'v0_p2wpkh':
            outputs.append(output_from_options(bytes.fromhex(vout.scriptpubkey), vout.value))
            # print(output_from_options(bytes.fromhex(vout.scriptpubkey), vout.value).hex())
        
    # Compute the commitment hash
    message = get_commitment_hash(inpt, script_code, value_sats, outputs, transaction.locktime.to_bytes(4, byteorder="little"), transaction.vin[0].sequence.to_bytes(4, byteorder="little"))

    witnesses = []
    for vin in transaction.vin:
        witnesses.append(bytes.fromhex(vin.witness[0]))
    
    # verify
    verify(bytes.fromhex(transaction.vin[0].witness[1]), message, bytes.fromhex(transaction.vin[0].witness[0]))

    # Assemble
    input_script = input_from_utxo(bytes.fromhex(transaction.vin[0].txid), int(transaction.vin[0].vout))
    final = assemble_transaction([input_script], outputs, witnesses)

    # Reserialize without witness data and double-SHA256 to get the txid
    txid = get_txid([input_script], outputs)
    print(txid)
    print(filename)

    count += 1
    if count == 1:
        break

verification fails
097288d268f4749878cd4376d1c9aa78eb05541d9efe3b25b7500bfc5626ee74
7cd041411276a4b9d0ea004e6dd149f42cb09bd02ca5dda6851b3df068749b2d


In [9]:
# check that the input is higher than the output value
# check locktime
# check that the 

In [22]:
# verify
def verify_p2wpkh(transaction: object, prev_txn: object): 
    inpt = bytes.fromhex(transaction.txid)[::-1] + int(prev_txn.vout).to_bytes(4, byteorder='little')
    value_sats = prev_txn.prevout.value
    script_code = get_p2wpkh_scriptcode(bytes.fromhex(prev_txn.prevout.scriptpubkey)[2:])

    outputs = []
    for vout in transaction.vout:
        if vout.scriptpubkey_type == 'v0_p2wpkh':
            outputs.append(output_from_options(bytes.fromhex(vout.scriptpubkey), vout.value))
            # print(output_from_options(bytes.fromhex(vout.scriptpubkey), vout.value).hex())
        
    # Compute the commitment hash
    message = get_commitment_hash(inpt, script_code, value_sats, outputs, transaction.locktime.to_bytes(4, byteorder="little"), prev_txn.sequence.to_bytes(4, byteorder="little"))
    
    # Print the commitment hash for each transaction
    # print(f"Transaction: {filename}, Commitment Hash: {message.hex()}")

    # verify
    verify(bytes.fromhex(prev_txn.witness[1]), message, bytes.fromhex(prev_txn.witness[0]))

    # Assemble
    # final = assemble_transaction([input_script], [multisig_output, change_output], witnesses)

    # Reserialize without witness data and double-SHA256 to get the txid
    # txid = get_txid([input_script], [multisig_output, change_output])


In [23]:
def verify_p2wsh(transaction: object, prev_txn: object): 
    pass

def verify_p2sh(transaction: object, prev_txn: object): 
    pass

def verify_p2tr(transaction: object, prev_txn: object): 
    pass

def verify_p2pkh(transaction: object, prev_txn: object): 
    pass



In [30]:
count = 0
# Iterate over each transaction 
# verify_p2wpkh
for filename, transaction in transactions.items():

    for prev_txn in transaction.vin:
        if prev_txn.prevout.scriptpubkey_type == 'v0_p2wpkh':
            # print('verifying v0_p2wpkh')
            verify_p2wpkh(transaction, prev_txn)
        if prev_txn.prevout.scriptpubkey_type == 'v0_p2wsh':
            verify_p2wsh(transaction, prev_txn)
        if prev_txn.prevout.scriptpubkey_type == 'p2sh':
            verify_p2sh(transaction, prev_txn)
        if prev_txn.prevout.scriptpubkey_type == 'v1_p2tr':
            verify_p2tr(transaction, prev_txn)
        if prev_txn.prevout.scriptpubkey_type == 'p2pkh':
            verify_p2pkh(transaction, prev_txn)

    input_script = []
    witnesses = []
    for prev_txn in transaction.vin:
        witnesses.append(bytes.fromhex(prev_txn.witness[0]))
        input_script.append(input_from_utxo(bytes.fromhex(transaction.vin[0].txid), int(transaction.vin[0].vout)))
   
    final = assemble_transaction(input_script, outputs, witnesses)

    # Reserialize without witness data and double-SHA256 to get the txid
    txid = get_txid(input_script, outputs)
    print(txid)
    print(filename)

    count += 1
    if count == 10:
        break

verification fails


TypeError: sequence item 0: expected a bytes-like object, list found

In [15]:
# BUILD BLOCK



In [16]:
def calculate_merkle_root(txids):
    """
    Calculate the Merkle Root from a list of transaction IDs (TXIDs).

    Parameters:
    - txids (list of str): List of transaction IDs (TXIDs) in reverse byte order.

    Returns:
    - str: The calculated Merkle Root.
    """
    # Convert the list of TXIDs to bytes in natural byte order
    txid_bytes = [bytes.fromhex(txid)[::-1] for txid in txids]

    # Helper function to calculate the hash of two concatenated inputs
    def hash_pair(a, b):
        return hashlib.sha256(hashlib.sha256(a + b).digest()).digest()

    # If there's only one TXID, return its hash as the Merkle Root
    if len(txid_bytes) == 1:
        return txid_bytes[0].hex()

    # Initialize the list of hashes with the TXIDs
    hashes = txid_bytes[:]

    # Calculate the Merkle Root iteratively
    while len(hashes) > 1:
        # If the number of hashes is odd, duplicate the last one
        if len(hashes) % 2 != 0:
            hashes.append(hashes[-1])
        
        # Calculate the hash of each pair of hashes
        hashes = [hash_pair(hashes[i], hashes[i + 1]) for i in range(0, len(hashes), 2)]

    # Return the Merkle Root as a hexadecimal string
    return hashes[0].hex()



In [ ]:
def build_block(version, prev_block_hash, merkle_root, time, bits, nonce, transactions):
    """
    Build a block according to the given specifications.

    Parameters:
    - version (int): The version number for the block.
    - prev_block_hash (bytes): The block hash of a previous block this block is building on top of.
    - merkle_root (bytes): A fingerprint for all of the transactions included in the block.
    - time (int): The current time as a Unix timestamp.
    - bits (int): A compact representation of the current target.
    - nonce (int): The nonce value for the block.
    - transactions (bytes): All of the raw transactions included in the block concatenated together.

    Returns:
    - bytes: The binary representation of the block.
    """
    # Concatenate the fields into binary format
    block_data = (
        version.to_bytes(4, byteorder='little') +
        prev_block_hash +
        merkle_root +
        time.to_bytes(4, byteorder='little') +
        bits.to_bytes(4, byteorder='little') +
        nonce.to_bytes(4, byteorder='little') +
        len(transactions).to_bytes(1, byteorder='little') +
        transactions
    )

    return block_data

def build_coinbase_transaction(block_height, block_reward, transaction_fees, coinbase_data):
    # Step 1: Create transaction inputs
    coinbase_txin = {
        "prev_txid": "0" * 64,  # Blank TXID
        "prev_vout": 0xffffffff,  # Maximum value for VOUT
        "script_sig": block_height.to_bytes(4, 'little') + coinbase_data  # Add block height and custom data
    }

    # Step 2: Create transaction outputs
    # Output 1: Block reward
    coinbase_txout_1 = {
        "value": block_reward,
        "script_pubkey": "output_script_1"
    }
    # Output 2: Transaction fees
    coinbase_txout_2 = {
        "value": transaction_fees,
        "script_pubkey": "output_script_2"
    }

    # Step 3: Construct the coinbase transaction
    coinbase_tx = {
        "version": 1,
        "inputs": [coinbase_txin],
        "outputs": [coinbase_txout_1, coinbase_txout_2],
        "locktime": 0
    }

    return coinbase_tx

In [17]:
# Example usage:
block_height = 227836
block_reward = 5000000000  # 50 BTC (in satoshis)
transaction_fees = 1000000  # Example transaction fees collected (in satoshis)
coinbase_data = b"btc_cpr"

coinbase_tx = build_coinbase_transaction(block_height, block_reward, transaction_fees, coinbase_data)

print("Coinbase transaction:")
print(coinbase_tx)



txids = []
for filename, transaction in transactions.items():
    txids.append(filename)

merkle_root = calculate_merkle_root(txids)
print("Merkle Root:", merkle_root)



# Example usage:
# Pretend data for the block
version = 1
prev_block_hash = bytes.fromhex("0000000000000000000000000000000000000000000000000000000000000000")
# merkle_root = bytes.fromhex("aabbccddeeff00112233445566778899aabbccddeeff00112233445566778899")
time = 1639707600  # Unix timestamp for 2021-12-17 00:00:00 UTC
bits = 0x1c0ae493
nonce = 12345
transactions = b"transaction_data_1transaction_data_2transaction_data_3"  # Example raw transactions

# Build the block
block = build_block(version, prev_block_hash, merkle_root, time, bits, nonce, transactions)

print("Block data:", block.hex())






Merkle Root: 79545390bd829f2c4e2ff573e15d5b0285d08f5b35567907e0f844b3040f69de


Coinbase transaction:
{'version': 1, 'inputs': [{'prev_txid': '0000000000000000000000000000000000000000000000000000000000000000', 'prev_vout': 4294967295, 'script_sig': b'\xfcy\x03\x00btc_cpr'}], 'outputs': [{'value': 5000000000, 'script_pubkey': 'output_script_1'}, {'value': 1000000, 'script_pubkey': 'output_script_2'}], 'locktime': 0}


In [ ]:
# for p2wpkh in the chain code class so once you have that part you can get 
# the witness and the public key from the raw transaction data and use the 
# ecdsa module to verify the signature. 

In [23]:
from ecdsa import VerifyingKey, SECP256k1

In [24]:
def verify_p2wpkh(txn):
    """
    Verify P2WPKH transactions by checking the signature using ECDSA.

    Parameters:
    - txn (Txn): The transaction object.

    Returns:
    - bool: True if the transaction is valid, False otherwise.
    """
    for vin in txn.vin:
        if vin.prevout.scriptpubkey_type == "p2wpkh":
            witness = vin.witness
            if len(witness) != 2:
                print(f"Invalid witness data for transaction {txn.txid}")
                return False

            signature = witness[0]
            public_key = witness[1]

            try:
                # Convert the public key to a verifying key
                verifying_key = VerifyingKey.from_string(bytes.fromhex(public_key), curve=SECP256k1)
                # Verify the signature
                is_valid = verifying_key.verify(bytes.fromhex(signature), bytes.fromhex(txn.txid))
                if not is_valid:
                    print(f"Invalid signature for transaction {txn.txid}")
                    return False
            except Exception as e:
                print(f"Error verifying P2WPKH transaction {txn.txid}: {e}")
                return False

    return True

In [31]:
def verify_p2wsh(txn):
    """
    Verify P2WSH transactions by checking the script using ECDSA.

    Parameters:
    - txn (Txn): The transaction object.

    Returns:
    - bool: True if the transaction is valid, False otherwise.
    """
    for vin in txn.vin:
        if vin.prevout.scriptpubkey_type == "p2wsh":
            witness = vin.witness
            if len(witness) < 1:
                print(f"Invalid witness data for transaction {txn.txid}")
                return False

            script = witness[-1]  # Last item in witness is the redeem script

            try:
                # Here you would execute your script validation logic
                # For simplicity, let's assume it always returns True
                is_valid_script = True
                if not is_valid_script:
                    print(f"Invalid script for transaction {txn.txid}")
                    return False
            except Exception as e:
                print(f"Error verifying P2WSH transaction {txn.txid}: {e}")
                return False

    return True


In [32]:
# Loop through transactions
for txid, txn in transactions.items():
    # print(f"Verifying transaction {txid}...")
    # if verify_p2wpkh(txn):
    #     print(f"Transaction {txid} is valid.")
    # else:
    #     print(f"Transaction {txid} is not valid.")
    if not verify_p2wpkh(txn):
        print(f"Transaction {txid} is not valid.")
    if not verify_p2wsh(txn):
        print(f"Transaction {txid} is not valid.")

In [36]:
count = 0
for txid, txn in transactions.items():
    for vout in txn.vout:
        if vout.scriptpubkey_type == "unknown":
            # print(f"Transaction {txid} has unknown type.")
            count += 1
            break
print(f"count {count}")


count 42


In [22]:
sorted_transactions = sorted(transactions, key=lambda tx: tx.fee, reverse=True)
print("Sorted Transactions by Fee Descending:", sorted_transactions)

AttributeError: 'str' object has no attribute 'fee'

In [10]:
class Block:
    def __init__(self):
        self.transactions = []
        self.total_fee = 0
        self.total_weight = 0

In [11]:
def branch_and_bound(tx_data, max_weight):
    best_block = Block()
    
    def backtrack(block, tx_indices):
        nonlocal best_block
        
        # Calculate current block's fee and weight
        block.total_fee = sum(tx.fee for tx in block.transactions)
        block.total_weight = sum(tx.weight for tx in block.transactions)
        
        # Update best block if current block has higher fee and within weight limit
        if block.total_fee > best_block.total_fee and block.total_weight <= max_weight:
            best_block = block
        
        # Explore all possible transactions to add
        for tx in tx_data.values():
            if tx not in block.transactions:
                new_block = Block()
                new_block.transactions = block.transactions + [tx]
                backtrack(new_block, new_block.transactions)
    
    # Start with an empty block and explore all possible transactions to add
    backtrack(Block(), [])
    
    return best_block


In [12]:
# Usage example:
tx_data = read_directory("mempool")
max_weight = 4000000  # Assuming maximum block weight constraint
best_block = branch_and_bound(tx_data, max_weight)
print("Best Block Transactions:", best_block.transactions)
print("Total Fee:", best_block.total_fee)
print("Total Weight:", best_block.total_weight)


KeyboardInterrupt: 

In [12]:
def branch_and_bound(tx_data, max_weight):
    best_block = Block()
    stack = [(Block(), [])]  # Initialize stack with initial block and transactions
    
    while stack:
        block, tx_indices = stack.pop()  # Pop the block and transactions from the stack
        
        # Calculate current block's fee and weight
        block.total_fee = sum(tx.fee for tx in block.transactions)
        block.total_weight = sum(tx.weight for tx in block.transactions)
        
        # Update best block if current block has higher fee and within weight limit
        if block.total_fee > best_block.total_fee and block.total_weight <= max_weight:
            best_block = block
        
        # Explore all possible transactions to add
        for tx in tx_data.values():
            if tx not in block.transactions:
                new_block = Block()
                new_block.transactions = block.transactions + [tx]
                stack.append((new_block, new_block.transactions))
    
    return best_block


In [11]:
class Block:
    def __init__(self):
        self.transactions = []
        self.total_fee = 0
        self.total_weight = 0

def greedy_knapsack(tx_data, max_weight):
    # Sort transactions by fee/weight ratio in descending order
    sorted_tx = sorted(tx_data.values(), key=lambda tx: tx.fee / tx.weight, reverse=True)
    
    block = Block()
    for tx in sorted_tx:
        if block.total_weight + tx.weight <= max_weight:
            block.transactions.append(tx)
            block.total_fee += tx.fee
            block.total_weight += tx.weight
    
    return block


In [13]:
# Usage example:
tx_data = read_directory("mempool")
max_weight = 4000000  # 4 million weight units
best_block = greedy_knapsack(tx_data, max_weight)
# print("Best Block Transactions:", best_block.transactions)
print("Total Fee:", best_block.total_fee)
print("Total Weight:", best_block.total_weight)


Total Fee: 7629673
Total Weight: 3996479


In [14]:
class CoinbaseTransaction:
    def __init__(self, block_height, coinbase_data, reward, total_fees):
        self.block_height = block_height
        self.coinbase_data = coinbase_data
        self.reward = reward
        self.total_fees = total_fees
    
    def generate_transaction(self):
        # Create the input for the coinbase transaction
        coinbase_input = {
            "coinbase": self.coinbase_data,  # Arbitrary data
            "sequence": 0xffffffff  # Arbitrary value
        }
        
        # Calculate the total value for the coinbase transaction
        total_value = self.reward + self.total_fees
        
        # Create the output for the coinbase transaction
        coinbase_output = {
            "value": total_value,
            "scriptpubkey": "miner_address"  # Address where the miner receives the reward
        }
        
        # Create the coinbase transaction
        coinbase_txn = {
            "version": 1,  # Transaction version
            "locktime": 0,  # Locktime
            "vin": [coinbase_input],  # Input
            "vout": [coinbase_output]  # Output
        }
        
        return coinbase_txn

# Usage example:
block_height = 123456  # Example block height
coinbase_data = "Banks on the brink"  # Example coinbase data
reward = 6.25  # Block reward for Bitcoin (in BTC)
total_fees = 10.5  # Example total fees collected from transactions (in BTC)
coinbase_txn = CoinbaseTransaction(block_height, coinbase_data, reward, total_fees).generate_transaction()
print("Coinbase Transaction:", coinbase_txn)


Coinbase Transaction: {'version': 1, 'locktime': 0, 'vin': [{'coinbase': 'Banks on the brink', 'sequence': 4294967295}], 'vout': [{'value': 16.75, 'scriptpubkey': 'miner_address'}]}


In [19]:
import hashlib

class Block:
    def __init__(self, previous_hash, coinbase_transaction, transactions, nonce):
        self.previous_hash = previous_hash
        self.coinbase_transaction = coinbase_transaction
        self.transactions = transactions
        self.nonce = nonce

    def compute_hash(self):
        block_header = self.previous_hash + str(self.transactions) + str(self.nonce)
        return hashlib.sha256(block_header.encode()).hexdigest()

def proof_of_work(block, difficulty):
    target = "0" * difficulty  # Target value with leading zeros
    while True:
        block_hash = block.compute_hash()
        if block_hash.startswith(target):
            return block_hash
        else:
            block.nonce += 1  # Increment nonce and try again

# Example usage:
previous_hash = "0000000000000000000000000000000000000000000000000000000000000000"  # Genesis block hash
nonce = 0  # Initial nonce
difficulty = 4  # Example difficulty (number of leading zeros required in the hash)

# Create a block
block = Block(previous_hash, coinbase_txn, best_block.transactions, nonce)

# Perform PoW
result_hash = proof_of_work(block, difficulty)
print("PoW Result Hash:", result_hash)


PoW Result Hash: 0000ba105cc03c4c6d1274ede005ba86fc4eb8066db1d9586559db8587796604


Transaction: 7cd041411276a4b9d0ea004e6dd149f42cb09bd02ca5dda6851b3df068749b2d, Commitment Hash: 5b9362899340b1d50237ed8bc7a21bf76bd129ce94f222baa7f0653d73305f49
Transaction: c990d29bd10828ba40991b687362f532df79903424647dd1f9a5e2ace3edabca, Commitment Hash: 0212ea5387d3b5232aefecaaeaa1655a98ca3c59b0641dada5db994fb26f3b27
Transaction: 119604185a31e515e86ba0aec70559e7169600eab5adf943039b0a8b794b40df, Commitment Hash: 4420b4dbd4c51c0c1dd852edc02fc48f462b66836ec99d0ddb5fb069a67d7427
Transaction: c3576a146165bdd8ecbfc79f18c54c8c51abd46bc0d093b01e640b6692372a93, Commitment Hash: ddfdae4fbf7fa6694aefeb1fea8cfccffb0b3cdc2ffb2f4550359705c3e1e78b
Transaction: 9fbc187e552b9e93406df86a4ebac8b67ccc0c4c321d0297edd8ffb87d4f5a45, Commitment Hash: a1463b92339e14805d23dcce8e74fceeb952899bc68557b777fad3a4155402a4
Transaction: 7de645056d100ee9d175ec61a90acc3d67812f93a3dae605a94f4db0f7c2a153, Commitment Hash: 58793bb12597bcb9630ae67775f6db708fd9470ed0b035f5069a5a8875d0fa6e
Transaction: 19faba5514956cde3a721

In [ ]:
class Block: 
    def __init__(self, data, previous_hash): 
        """ 
        Initializes a new Block object with  
        the given data and previous hash. 
        """
        self.data = data 
        self.previous_hash = previous_hash 
        self.nonce = 0
        self.hash = self.calculate_hash() 
  
    def calculate_hash(self): 
        """ 
        Calculates the SHA-256 hash of the  
        block's data, previous hash, and nonce. 
        """
        sha = hashlib.sha256() 
        sha.update(str(self.data).encode('utf-8') + 
                   str(self.previous_hash).encode('utf-8') + 
                   str(self.nonce).encode('utf-8')) 
        return sha.hexdigest() 
  
    def mine_block(self, difficulty): 
        """ 
        Mines the block using the Proof-of-Work algorithm  
        with the given difficulty level. 
        """
        while self.hash[0:difficulty] != "0" * difficulty: 
            self.nonce += 1
            self.hash = self.calculate_hash() 
  
        print("Block mined:", self.hash) 

Merkle Root: 79545390bd829f2c4e2ff573e15d5b0285d08f5b35567907e0f844b3040f69de


In [ ]:
def vin_transaction_types(transaction):
    """This function extracts the transaction types."""
    # Define a set to store transaction types
    txn_typs = set()

    # Iterate through each transaction in the mempool
    for filename, transaction in transactions.items():
        for vin in transaction.vin:
            txn_typs.add(vin.prevout.scriptpubkey_type)

    # Print the list of transaction types
    print("Vin Transaction Types:")
    for txn_type in txn_typs:
        print(txn_type)

    return txn_typs

def vout_transaction_types(transaction):
    """This function extracts the transaction types."""
    # Define a set to store transaction types
    txn_typs = set()

    # Iterate through each transaction in the mempool
    for filename, transaction in transactions.items():
        for vout in transaction.vout:
            txn_typs.add(vout.scriptpubkey_type)

    # Print the list of transaction types
    print("Vout Transaction Types:")
    for txn_type in txn_typs:
        print(txn_type)

    return txn_typs

txn_types = vout_transaction_types(transactions)
print("")
txn_types = vin_transaction_types(transactions)